# qSonify Hello World: Hearing Algorithms to Aid Algorithmic Abstraction
#### https://github.com/jiosue/qSonify

We will show how to use qSonify to recognize patterns in quantum algorithms in order to help us understand their decomposition and/or abstraction. *qSonify required numpy and midiutils*

In [1]:
import qSonify

For our example, let's define two algorithms. Firstly, qSonify gives the ability to represent certain supported gates as strings, for example "cx(0, 2)" for a controlled-not gate from qubit 0 to 2. But for unitary transformation which are not by default supported, we can instead input a qSonify.Gate object, which takes in a unitary matrix and a tuple of qubits the the unitary will be applied to. An algorithm is a list of these gates, either string gates or qSonify.Gate objects.

For example, a Hadamard gate on qubit 2 can be created in any of the following ways (let n = 1 / sqrt(2)):

    H2 = qSonify.gates.H(2)
    H2 = "H(2)"
    H2 = qSonify.Gate([[n, n], [n, -n]], (2,))
    
For simple, common gates this is redundant. However, for more general unitary transformations, the first two will not work. For examle, consider a gate operating on qubits 1 and 3 with some arbitrary 4x4 unitary matrix U, ie something of the form:

    U = [[_, _, _, _],
         [_, _, _, _],
         [_, _, _, _],
         [_, _, _, _]]
     
 This gate can only be represented as:
 
     G = qSonify.Gate(U, (1, 3))
     
Note that when using qSonify.Gate, it will NOT check to be sure that the matrix you provide is indeed unitary.
     
Now let's define our two algorithms.

In [2]:
alg1 = ["h(0)", "cx(0, 1)", "cx(0, 2)", "cx(0, 3)", "cx(0, 4)"]
alg2 = ['h(0)', 'rz(pi, 2)', qSonify.Gate([[0.5, 0.75**0.5], [-0.75**0.5, 0.5]], (4,)), 'rx(pi/2, 2)', 'cx(0, 3)']

Next we have to choose a mapping from output to midi file. Many mappings require us to choose a set of notes to use, let's define ours here.

In [3]:
notes = ("c4", "d4", "e4", "f4", "g4", "a4", "b4", "c5")

Now let's define/choose our mapping. We can either define our own mapping or choose one of the one's already implemented. Choosing a mapping is really the essence of sonification. I have chosen to map the outputs to MIDI notes because it allows for simple implementation. I encourage anyone to try out all these mappings and define your own, as well as try mappings to representations other than MIDI (not supported by qSonify :( ). I have found the grandpiano mapping to sound quite nice on this particular example.

In [4]:
#mapping = qSonify.maps.fermionic(notes=notes)
#mapping = qSonify.maps.scale(notes=notes)
mapping = qSonify.maps.grandpiano(low_notes=("c3", "d3", "e3", "f3", "g3"), high_notes=notes)
#mapping = qSonify.maps.stringquartet()
#mapping = qSonify.maps.frequencymapping(low_freq=300, base=4)

# user_defined mapping must be of this form.
# as an examle, this particular user defined mapping is
# the same as qSonify.frequencymapping(low_freq=300, base=4).
# To see more details of how to add more tracks, etc, see the
# `maps` folder.
# def mapping(res, name, tempo):
#     """
#     res: list of strings, outputs of the quantum computer run, ie
#          ['10010', '01101', ...]
#     name: str, name of the song.
#     tempo: int, tempo of the song.
#     return: qSonify.Song object.
#     """
#     s = qSonify.Song(name=name, tempo=tempo, num_tracks=1)
#     for x in res:
#         note = qSonify.freq_to_note(int(x, base=4) + 300)
#         s.addNote(note, duration=.5) # eigth notes
#     return s

Now let's run it! We use the function with the following keyword arguments:

    def alg_to_song(algorithm, num_qubits=None, num_samples=40, mapping=maps.default_map, name="alg", tempo=100):
        """
        Make a song from an algorithm. Markovian sample the algorithm, then map
        to a Song object.

        algorithm: list of Gate objects and/or string gates. Example:
                      ["cx(0, 1)", 
                       "x(0)", 
                       qSonify.Gate(unitary=[[...], ...], qubits=(1, 2))
                      ]
        num_qubits: int, number of qubits to run each algorithm on. If num_qubits
                         is None, then it will run on the minimum required.
        num_samples: int, number of samples to take from the quantum computer,
                          for most mappings this is equal to the number of beats.
        mapping: function, which mapping from output to sound to use. Can either 
                           use the ones already defined (ie maps.[map name](args)), 
                           or create your own mapping function. mapping must take
                           a list of outputs from the qc, a name of the song, and
                           a tempo of the song, and return a Song object.
        name: str, name of song.
        tempo: int, tempo of song.

        returns: qSonify.Song object
        """

Markovian sampling is inputting the output of the previous run in as the input to the next run. This lets us explore more of the transformation.

In [5]:
kwargs = dict(num_samples=40, mapping=mapping, tempo=150)

s1 = qSonify.alg_to_song(alg1, name="HelloWorld_alg1", **kwargs)
s2 = qSonify.alg_to_song(alg2, name="HelloWorld_alg2", **kwargs)

Now let's listen to them! *Note the play function only works on Windows right now, because it just calls the system to open the midi file, which Windows by default opens in Windows Media Player. To play the files on your computer if it is non Windows, go to the folder output/ and open the saved .mid file however you want.*

In [7]:
s1.play()

In [55]:
s2.play()

Now let's see if we can still "hear" these algorithms when they are embedded within another circuit.

In [6]:
alg1_emb = ["h(4)", "rz(pi/8, 2)"] + alg1 + ["h(1)"]
alg2_emb = ["h(2)", "rz(pi/4, 0)"] + alg2 + ["h(0)"]
s1_emb = qSonify.alg_to_song(alg1_emb, name="HelloWorld_alg1_emb", **kwargs)
s2_emb = qSonify.alg_to_song(alg2_emb, name="HelloWorld_alg2_emb", **kwargs)

In [57]:
s1_emb.play() # compare to s1.play()

In [58]:
s2_emb.play() # compare to s2.play()

Can you hear the similarities? Do you hear the original algorithm embedded within the new one? Now let's try one more thing. Let see if we can hear both algorithm1 and algorithm2 when they are performed in the same algorithm and embedded within other gate sequences.

In [50]:
alg12_emb = alg1_emb + alg2_emb + alg2_emb + alg1_emb
s12_emb = qSonify.alg_to_song(alg12_emb, name="HelloWorld_alg12_emb", **kwargs)

Now let's listen to it. Try and compare `s12_emb` to `s1`, `s2`, `s1_emb`, and `s2_emb`, and try to see if you can hear the original `alg1` and `alg2`.

In [51]:
s12_emb.play()

So did you hear it? Sonifying quantum algorithms in this way could help researchers "decompile" algorithms that are represented in terms of elementary gate operations that are generally hard to abstract from. Try testing some different mappings or defining your own! All the midi file outputs from `s.play()` or `s.writeFile()` are saved in the "output" folder in this same directory.

I encourage anyone to use a software to convert the MIDI files into sheet music and check it out. Do you think it is easier to "see" the embedded algorithms or "hear" them? Sonification is a powerful tool because our ears can often hear patterns that are difficult for our eyes to pick out.